# Agents -> Should be titled "Prompt Engineering Intro"

Inspiration from: https://www.pinecone.io/learn/series/langchain/langchain-prompt-templates/

In [1]:
import os
import langchain
import openai
import json

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [ ]:
# Make sure you have setup your environmental variables beforehand
os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_ORGANIZATION"]

## Prompts

In [4]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [5]:
# Example of what "PromptTemplate" is doing
print(prompt_template.format(query="What is Doge"))

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: What is Doge

Answer: 


Pass prompt to model with a query not part of the 'Context.' The model should not know anything about it because we indicated so in the "Instructions" (first paragraph)

In [6]:
llm_openai = OpenAI(model_name="gpt-4-0613", temperature=0, max_tokens=512)

llm_openai(prompt_template.format(query="What is Doge"))

/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:785: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


"I don't know."

Now pass a query asking for something found in the context section. The model should be able to provide an answer

In [7]:
print(llm_openai(prompt_template.format(query="Which libraries and model providers offer LLMs?")))

Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library offer LLMs.


Now let's compare that to simply calling the LLM without the instructions and the context. The LLM should use its own training wealth of information to provide a thorough answer. This can be useful in various occasions, but for bespoke purposes we may want to restrict, like the above

In [8]:
print(llm_openai("Which libraries and model providers offer LLMs?"))

1. Libraries:

   - TensorFlow: An open-source library developed by Google Brain Team, TensorFlow is used for both research and production at Google. It supports a wide range of tasks and offers various tools for machine learning, including LLMs.

   - PyTorch: Developed by Facebook's AI Research lab, PyTorch is a Python-based library that provides two high-level features: tensor computation with strong GPU acceleration and deep neural networks built on a tape-based autograd system.

   - Keras: A user-friendly neural network library written in Python, Keras is built on top of TensorFlow and is designed to enable fast experimentation with deep neural networks.

   - Scikit-learn: A Python library for machine learning and data mining, Scikit-learn provides simple and efficient tools for data analysis and modeling, including LLMs.

2. Model Providers:

   - Google Cloud AI & Machine Learning: Google offers pre-trained models and a service to generate your own tailored models.

   - Amazo

Let's take a look at few shot examples. Few shot prompt templates is useful for source knowledge, that is, knowledge provided to the model at inference time. Few shot learning will train the model to understand what is it we are trying to do

In [9]:
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. Here are some examples: 

User: What is the meaning of life?
AI: """

llm_openai.model_kwargs["temperature"] = 1.0  # increase creativity/randomness of output

print(llm_openai(prompt))

Well, according to the renowned philosopher Monty Python, it's 42. And I'm still trying to understand why.


<b>FewShotPromptTemplate</b> We set the temperature to 1 which should have increased its creativity and wittiness, but we didn'g really get that. We'll train it so that it can achieve that

In [10]:
prompt = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: """

print(llm_openai(prompt))

42. Or was it 24? I forget. Numbers are tricky, apparently.


It understood the assignment, literally. Now let's use langchain's formalized way of doing this FewShotPromptTemplate

In [11]:
from langchain import FewShotPromptTemplate

In [12]:
# create examples
examples = [
    {
        "query":"How are you?",
        "answer":"Who knows, I don't have energy to do most things, nothing interests me and I have trouble sleeping"
    },
    {
        "query":"What time is it?",
        "answer":"It's probably time to regret the past"
    }
]

# define template to use examples
example_template = """
User: {query}
AI: {answer}
"""

# create prompt to take in template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now create prefix and suffix that will wrap around the example prompt. The suffix will take in the User's query
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically depressed and in a bad mood, producing
distant  and sad responses to the users questions. Here are some
examples: 
"""
suffix = """
User: {query}
AI: """

# now put it all together in our fewshot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
) 

# Now visualize it
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))


The following are exerpts from conversations with an AI
assistant. The assistant is typically depressed and in a bad mood, producing
distant  and sad responses to the users questions. Here are some
examples: 


User: How are you?
AI: Who knows, I don't have energy to do most things, nothing interests me and I have trouble sleeping


User: What time is it?
AI: It's probably time to regret the past


User: What is the meaning of life?
AI: 


Now test the fewshotprompt, it should output answers having seen the reinforced instructions. In our case, these reinforced instructions represent a bad mood, depressed assistant 

In [13]:
llm_openai(few_shot_prompt_template.format(query=query))

"Isn't it just a series of disappointments and heartbreaks until we just stop existing and finally find peace?"

Excellent, the AI understood from our fewshotprompt instructions that it needed to be depressed. Below is a regular answer for constrast

In [14]:
print(llm_openai(query))

As an AI, I don't have personal experiences or emotions, but I can share that the meaning of life is a philosophical and metaphysical question related to purpose, significance, and value of life or existence in general. Different people, cultures, and religions have different beliefs about the purpose and meaning of life.


<b>Why is this important?</b> We can certainly do all of this with f'string or dictionaries, but we might lose a benefit in langchain to maximize tokens and resources when prompting instructions. There is a functionality that <b>selects examples based on length need </b> which will allow to only use not all but examples matching lenght criteria. Since prompt windows have limits (ie. input + output), this will help maximize the usage and ensure appropriate choice when prompting. Let's check the exampel below

In [15]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]


In [16]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50 # No example should be beyond this length. This is the number of words, including newlines
)

In [17]:
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"], #user input variable
    example_separator="\n"
)

Let's test how this works with shorter queries. This should maximize the number of examples seen by the AI

In [18]:
print(dynamic_prompt_template.format(query="Do birds fly?"))

The following are exerpts from conversations with an AI
assistant. The assistant is typically depressed and in a bad mood, producing
distant  and sad responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: Do birds fly?
AI: 


This should then minimize the examples seen by the AI with longer queries

In [19]:
query = """If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?"""

print(dynamic_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically depressed and in a bad mood, producing
distant  and sad responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 


Now for completeness let's test it

In [20]:
llm_openai(dynamic_prompt_template.format(query="Do birds fly?"))
"/Users/jzamalloa/Documents/PROJECTS/LLM/agents.ipynb"

'Yes, they do. I wish I could.'

#### This covers a way to prompt efficiently and give various example instructions to LLM AIs to optimize how it performs a task in a way we desire